In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from epiweeks import Week
from datetime import date, datetime
from os.path import dirname, join, isfile, isdir

from bulletin.commom import static
from bulletin.commom.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf, normalize_igbe
from bulletin.commom.utils import isvaliddate

from bulletin.commom.utils import Timer, auto_fit_columns
output = join("output")

In [2]:
municipios = static.municipios.loc[static.municipios['uf']=='PR',['ibge','municipio','populacao']].copy()
municipios['ibge'] = municipios['ibge'].apply(str)

regionais = static.regionais[['ibge','nm_macro','nu_reg','nm_reg']].copy()
regionais['ibge'] = regionais['ibge'].apply(str)
regionais = regionais.rename(columns={'nu_reg':'rs'})

populacao_macro = pd.merge(regionais,municipios,on='ibge',how='inner').groupby('nm_macro')[['populacao']].sum()
populacao_rs = pd.merge(regionais,municipios,on='ibge',how='inner').groupby('rs')[['populacao']].sum()
populacao_mun = municipios

def ajust_table(table,tipo='casos'):
    return table.unstack(level=[0,1,2]).swaplevel(0, 1, axis=1).swaplevel(1, 2, axis=1).swaplevel(2, 3, axis=1).fillna(0).astype(int).xs(tipo, axis=1, level= 3)

In [3]:
casos = pd.read_excel('input/casos_obitos.xlsx','casos',converters={
        'ibge': normalize_igbe
    }
)

In [4]:
obitos = pd.read_excel('input/casos_obitos.xlsx','obitos',converters={
        'ibge': normalize_igbe
    }
)

In [5]:
casos = pd.merge(left=casos, right=regionais, how='left', on='ibge')
casos = pd.merge(left=casos, right=municipios, how='left', on='ibge')
casos['ano'] = casos.apply(lambda row: Week.fromdate(row['dt_diag']).year, axis=1)
casos['mes'] = casos.apply(lambda row: Week.fromdate(row['dt_diag']).startdate().month, axis=1)
casos['se'] = casos.apply(lambda row: Week.fromdate(row['dt_diag']).week, axis=1)

In [6]:
obitos = pd.merge(left=obitos, right=regionais, how='left', on='ibge')
obitos = pd.merge(left=obitos, right=municipios, how='left', on='ibge')
obitos['ano'] = obitos.apply(lambda row: Week.fromdate(row['data_obito']).year, axis=1)
obitos['mes'] = obitos.apply(lambda row: Week.fromdate(row['data_obito']).startdate().month, axis=1)
obitos['se'] = obitos.apply(lambda row: Week.fromdate(row['data_obito']).week, axis=1)

In [7]:
casos_macros_se = casos.groupby(['ano','mes','se','nm_macro'])[['dt_diag']].count().rename(columns={'dt_diag':'casos'})
casos_regionais_se = casos.groupby(['ano','mes','se','rs','nm_reg'])[['dt_diag']].count().rename(columns={'dt_diag':'casos'})
casos_se = casos.groupby(['ano','mes','se','nm_macro','rs','nm_reg','ibge','municipio'])[['dt_diag']].count().rename(columns={'dt_diag':'casos'})

casos_macros_se.index.names = ['Ano','Mês','Semana Epidemiológica','Macro']
casos_regionais_se.index.names = ['Ano','Mês','Semana Epidemiológica','RS','Regional']
casos_se.index.names = ['Ano','Mês','Semana Epidemiológica','Macro','RS','Regional','Ibge','Municipio']

casos_macros_se = ajust_table(casos_macros_se,'casos')
casos_regionais_se = ajust_table(casos_regionais_se,'casos')
casos_se = ajust_table(casos_se,'casos')

In [8]:
obitos_macros_se = obitos.groupby(['ano','mes','se','nm_macro'])[['data_obito']].count().rename(columns={'data_obito':'obitos'})
obitos_regionais_se = obitos.groupby(['ano','mes','se','rs','nm_reg'])[['data_obito']].count().rename(columns={'data_obito':'obitos'})
obitos_se = obitos.groupby(['ano','mes','se','nm_macro','rs','nm_reg','ibge','municipio'])[['data_obito']].count().rename(columns={'data_obito':'obitos'})

obitos_macros_se.index.names = ['Ano','Mês','Semana Epidemiológica','Macro']
obitos_regionais_se.index.names = ['Ano','Mês','Semana Epidemiológica','RS','Regional']
obitos_se.index.names = ['Ano','Mês','Semana Epidemiológica','Macro','RS','Regional','Ibge','Municipio']

obitos_macros_se = ajust_table(obitos_macros_se,'obitos')
obitos_regionais_se = ajust_table(obitos_regionais_se,'obitos')
obitos_se = ajust_table(obitos_se,'obitos')

In [9]:
writer = pd.ExcelWriter(join(output,"casos_se.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casos_macros_se.to_excel(writer,sheet_name='casos_macros_se')
casos_regionais_se.to_excel(writer,sheet_name='casos_regionais_se')
casos_se.to_excel(writer,sheet_name='casos_se')

worksheet = writer.sheets[f"casos_macros_se"]
auto_fit_columns(worksheet,casos_macros_se)
worksheet = writer.sheets[f"casos_regionais_se"]
auto_fit_columns(worksheet,casos_regionais_se)
worksheet = writer.sheets[f"casos_se"]
auto_fit_columns(worksheet,casos_se)

obitos_macros_se.to_excel(writer,sheet_name='obitos_macros_se')
obitos_regionais_se.to_excel(writer,sheet_name='obitos_regionais_se')
obitos_se.to_excel(writer,sheet_name='obitos_se')

worksheet = writer.sheets[f"obitos_macros_se"]
auto_fit_columns(worksheet,obitos_macros_se)
worksheet = writer.sheets[f"obitos_regionais_se"]
auto_fit_columns(worksheet,obitos_regionais_se)
worksheet = writer.sheets[f"obitos_se"]
auto_fit_columns(worksheet,obitos_se)

writer.save()
writer.close()